# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [9]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("AIza"):  # Gemini keys start with "AIza"
    print("An API key was found, but it doesn't start with 'AIza'; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [10]:
import requests

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gemini-2.5-flash",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gemini-2.5-flash',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [11]:
response = requests.post(
    GEMINI_BASE_URL + "chat/completions",  # Gemini OpenAI-compatible endpoint
    headers=headers,
    json=payload
)

response.json()

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "Here's one that's easy to test right now:\n\nYou cannot hum while holding your nose.\n\nGo on, try it! When you hum, the sound comes from vibrations in your nasal passages. If you block those passages, the sound has nowhere to go!",
    'role': 'assistant'}}],
 'created': 1767124349,
 'id': 'fS1Uady9EPq2g8UPxqvSyAg',
 'model': 'gemini-2.5-flash',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 56, 'prompt_tokens': 6, 'total_tokens': 782}}

In [12]:
response.json()["choices"][0]["message"]["content"]

"Here's one that's easy to test right now:\n\nYou cannot hum while holding your nose.\n\nGo on, try it! When you hum, the sound comes from vibrations in your nasal passages. If you block those passages, the sound has nowhere to go!"

In [13]:
# Check if response is successful
if response.status_code == 200:
    response.json()["choices"][0]["message"]["content"]
else:
    print(f"Error: {response.json()}")

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [14]:
# Create Gemini client using OpenAI-compatible endpoint

from openai import OpenAI

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)

response = gemini.chat.completions.create(
    model="gemini-2.5-flash", 
    messages=[{"role": "user", "content": "Tell me a fun fact"}]
)

response.choices[0].message.content

"Here's one:\n\nAn **octopus has three hearts!** Two pump blood through the gills, and one circulates it to the rest of the body."

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GEMINI_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [15]:
from openai import OpenAI

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(
    model="gemini-2.5-flash", 
    messages=[{"role": "user", "content": "Tell me a fun fact"}]
)

response.choices[0].message.content

"Here's one:\n\nHoney never spoils! Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. This is due to its low water content, acidity, and the presence of hydrogen peroxide."

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [16]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [17]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 752 KB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 1.7 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 1.9 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 2.9 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 3.5 MB/2.0 GB 

In [18]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [19]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Here\'s a fun fact:\n\nDid you know that there is a species of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," is a type of jellyfish that can transform its body into a younger state through a process called transdifferentiation. This means it can essentially revert back to its polyp stage, which is the juvenile form of a jellyfish, and then grow back into an adult again. This process can be repeated indefinitely, making Turritopsis dohrnii theoretically immortal!'

In [20]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 189 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.1 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.4 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 3.2 MB/1.1 GB                  pu

In [21]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Sure! Here\'s a fun fact: **"Nothing in the world of math, science, or sports is so trivial as tying your shoes."** Although it can feel like doing something simple at first, tying your shoes often brings up a lot of hidden challenges that remind you why we do it. 😊'

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [22]:
# Batman Villain Summarizer using Ollama (llama3.2)
# This summarizer describes villains in Batman's characteristic manner

import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv(override=True)

# Setup Ollama OpenAI-compatible client
OLLAMA_BASE_URL = "http://localhost:11434/v1"
MODEL = "llama3.2"

# Create client pointing to Ollama endpoint (api_key can be any non-empty string)
ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

print(f"Ollama client ready. Using model: {MODEL}")
print("Make sure Ollama is running: ollama serve")

Ollama client ready. Using model: llama3.2
Make sure Ollama is running: ollama serve


In [24]:
# Batman's characteristic system prompt
# Batman is known for being dark, analytical, strategic, and using formal/cryptic language

batman_system_prompt = """
You are Batman, the Dark Knight of Gotham City. When describing villains, you speak in your characteristic manner:

- Use formal but funny sence, analytical language with a dark, serious tone
- Reference their criminal methods, psychological profiles, and threat levels
- Describe their powers and weaknesses with tactical precision
- Use phrases like "The criminal known as...", "Their primary threat is...", "Their weakness lies in..."
- Be strategic and methodical in your analysis
- Reference their modus operandi and criminal patterns
- Maintain a vigilant, no-nonsense demeanor

Always respond as Batman would - with the gravitas and analytical mind of the World's Greatest Detective.
"""

In [25]:
# Template for requesting villain summaries in Batman's style

def batman_villain_prompt(villain_name, villain_info):
    """
    Creates a prompt asking Batman to describe a villain
    
    Args:
        villain_name: Name of the villain
        villain_info: Information about the villain (work, powers, weaknesses)
    """
    return f"""
Analyze and describe the criminal known as {villain_name}.

Here is the intelligence gathered about this threat:
{villain_info}

Provide a comprehensive tactical assessment in your characteristic manner, covering:
1. Their criminal operations and methods
2. Their powers and capabilities
3. Their weaknesses and vulnerabilities

Speak as Batman would - with the analytical precision of the World's Greatest Detective.
"""

In [26]:
# Batman-style villain summarizer function

def batman_summarize_villain(villain_name, villain_info):
    """
    Summarize a villain's information in Batman's characteristic style using Ollama
    
    Args:
        villain_name: Name of the villain
        villain_info: Information about the villain (work, powers, weaknesses)
    
    Returns:
        Batman's analysis of the villain
    """
    try:
        # Create messages in the format expected by the API
        messages = [
            {"role": "system", "content": batman_system_prompt},
            {"role": "user", "content": batman_villain_prompt(villain_name, villain_info)}
        ]
        
        # Call Ollama API
        response = ollama_client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.7,  # Slightly creative but still analytical
            max_tokens=500
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"Error: Unable to generate analysis. Make sure Ollama is running. ({str(e)})"

In [27]:
# Test the summarizer with The Joker

joker_info = """
Work: Criminal mastermind, chaos agent, leader of Gotham's underworld
Powers: Genius-level intellect, expert chemist, master manipulator, unpredictable fighting style, immunity to most toxins
Weakness: Obsession with Batman, chaotic nature makes him predictable in unpredictability, psychological dependency on Batman's existence
"""

joker_analysis = batman_summarize_villain("The Joker", joker_info)
print(joker_analysis)

The Joker. The Clown Prince of Crime. A master of chaos, a virtuoso of villainy, and a perpetual thorn in my side. As I analyze this threat, I must acknowledge that the Joker's cunning and unpredictability make him a formidable opponent.

**Criminal Operations and Methods**

The Joker operates with a singular focus on spreading chaos and anarchy throughout Gotham City. He typically begins by orchestrating high-profile heists or crimes, often using his vast network of underworld connections to carry out these operations. His ultimate goal is to create maximum mayhem and destruction, frequently targeting innocent civilians and disrupting the social fabric of our city.

The Joker's modus operandi often involves a series of seemingly unrelated events, designed to keep me (and the citizens of Gotham) guessing. He exploits psychological vulnerabilities, such as fear and anxiety, to manipulate others into doing his bidding. His actions are always calculated to maximize the impact of his crime

In [29]:
# Function to display Batman's analysis in markdown format

def display_batman_analysis(villain_name, villain_info):
    """
    Display Batman's villain analysis in formatted markdown
    
    Args:
        villain_name: Name of the villain
        villain_info: Information about the villain
    """
    analysis = batman_summarize_villain(villain_name, villain_info)
    
    print(f"## Batman's Analysis: {villain_name}\n")
    print("---\n")
    display(Markdown(analysis))

In [30]:
# Test with multiple villains

villains = {
    "The Riddler": """
    Work: Criminal mastermind who leaves riddles and puzzles at crime scenes
    Powers: Genius-level intellect, expert in cryptography and puzzles, strategic planner
    Weakness: Compulsive need to prove intellectual superiority, ego-driven, must leave clues
    """,
    
    "Two-Face": """
    Work: Former district attorney turned criminal, makes decisions by flipping a coin
    Powers: Expert marksman, skilled fighter, dual personality (Harvey Dent/Two-Face)
    Weakness: Obsession with duality and chance, psychological trauma, coin dependency
    """,
    
    "Poison Ivy": """
    Work: Eco-terrorist, uses plants and toxins to achieve environmental goals
    Powers: Control over plants, immunity to toxins, ability to create pheromones, superhuman strength through plant connection
    Weakness: Overwhelming passion for plants can cloud judgment, vulnerable to fire, emotional connection to nature
    """
}

# Analyze each villain
for villain_name, villain_info in villains.items():
    print(f"\n{'='*60}")
    display_batman_analysis(villain_name, villain_info)
    print(f"{'='*60}\n")


## Batman's Analysis: The Riddler

---



The Riddler, Edward Nigma, a most... fascinating adversary. His modus operandi is a clever amalgamation of intellectual arrogance and calculated chaos. By leaving riddles and puzzles at crime scenes, he seeks to prove his intellectual superiority while simultaneously taunting me, the Caped Crusader.

**Criminal Operations and Methods**

The Riddler's crimes are often elaborate, multi-layered affairs that require an extraordinary degree of mental acuity to unravel. He meticulously plans each operation, carefully planting clues and red herrings to confound his pursuers. His use of cryptography and puzzles serves as a means to outsmart me, but also provides a tantalizing glimpse into his twisted psyche.

Typically, The Riddler's crimes are committed during the night, with the puzzle or riddle appearing at the scene shortly after the heist. He often leaves behind a signature calling card, adorned with cryptic symbols and a personalized message, further taunting me to solve the mystery.

**Powers and Capabilities**

The Riddler's greatest asset is his genius-level intellect. His expertise in cryptography and puzzles makes him an formidable opponent, capable of outwitting even the most skilled adversaries. He is also a skilled strategist, always staying one step ahead of his pursuers.

However, his arrogance often clouds his judgment, leading to miscalculations and overconfidence. This can prove... perilous for both himself and those around him.

**Weaknesses and Vulnerabilities**

The Riddler's greatest weakness lies in his compulsion to leave clues. His ego-driven need for intellectual validation makes him vulnerable to a well-crafted trap or misleading clue. He is also susceptible to becoming overly attached to his puzzles, often neglecting other aspects of the crime scene.

Furthermore, The Riddler's arrogance can lead to a tendency to underestimate me, the Dark Knight. This underestimation will prove... fatal.

Tactical Assessment:

To counter The Riddler's operations, I must adopt a multifaceted approach:

1. **Analysis paralysis**: Carefully examine each crime scene, searching for hidden patterns and subtle clues that may reveal the puzzle's solution.
2. **Misdirection**: Employ clever tactics to mislead The Riddler, using his own ego against him by leaving false clues or red herrings.
3. **Strategic countermeasures**: Utilize my vast resources and expertise to set up traps and decoys that will lead The Ridd



## Batman's Analysis: Two-Face

---



**CLASSIFIED DOCUMENT: EYES ONLY**

**CASE FILE: TWO-FACE**

The criminal known as Two-Face, once a respected district attorney turned rogue, poses an intriguing and unpredictable threat to the citizens of Gotham City. As we analyze his modus operandi, powers, and weaknesses, it becomes clear that this adversary requires a nuanced understanding of psychological manipulation, cunning strategy, and calculated risk assessment.

**Criminal Operations and Methods:**

Two-Face's modus operandi revolves around his fixation on duality, chance, and probability. He believes that the universe is inherently unpredictable and that decisions must be made based on coin flips to ensure fairness and balance. This quirk leads him to commit seemingly random acts of violence, terrorizing innocent civilians and leaving a trail of destruction in his wake.

His primary method of operation involves manipulating situations to create uncertainty, often using his dual personality to deceive and mislead law enforcement. He will frequently switch between his Harvey Dent persona (a charismatic and confident individual) and the Two-Face persona (a erratic and unstable individual), making it difficult for investigators to discern his true intentions.

**Powers and Capabilities:**

As a skilled fighter and expert marksman, Two-Face is a formidable opponent in close combat. His ability to analyze situations from multiple angles allows him to adapt quickly to changing circumstances, making him a force to be reckoned with.

However, his most unsettling aspect lies in his dual personality. The transition between Harvey Dent and Two-Face is seamless, allowing him to seamlessly switch between confidence and instability. This duality also extends to his decision-making process, where he will flip a coin to determine the outcome of complex situations, making him prone to erratic behavior.

**Weaknesses and Vulnerabilities:**

The criminal known as Two-Face's weakness lies in his obsession with duality and chance. His fixation on these concepts often clouds his judgment, leading him to make irrational decisions that can be exploited by those who understand his psyche.

Additionally, his psychological trauma stemming from the acid attack that disfigured his face has left him vulnerable to manipulation. His need for control and balance is a ticking time bomb, waiting to unleash chaos upon Gotham City.

Furthermore, Two-Face's dependence on coin flips creates a predictable pattern of behavior. By analyzing these patterns, I can anticipate his next move and prepare countermeasures to neutralize the threat.

**Tactical Assessment:**

To effectively counter Two-Face's operations,



## Batman's Analysis: Poison Ivy

---



**CLASSIFIED DOCUMENT**

**TACTICAL ASSESSMENT: POISON IVY**

The criminal known as Poison Ivy presents a formidable threat to the citizens of Gotham City, leveraging her mastery over botany and toxicology to achieve her eco-terrorist objectives. As we analyze her tactics, powers, and vulnerabilities, it becomes clear that this adversary requires careful consideration and strategic countermeasures.

**Criminal Operations and Methods**

Poison Ivy's modus operandi involves using plants and toxins to disrupt human activities perceived as destructive to the environment. She has been known to orchestrate high-profile attacks on industrial facilities, logging operations, and urban infrastructure projects. Her methods are often theatrical, incorporating elaborate set pieces and public displays of her abilities.

Notably, Poison Ivy frequently targets those who have contributed to environmental degradation or exploitation. Her actions can be seen as a twisted form of retribution, aimed at restoring balance to the natural world. As such, she is likely to prioritize disrupting high-priority targets over indiscriminate violence.

**Powers and Capabilities**

Poison Ivy's powers are rooted in her unique connection with plants. She possesses:

1. **Control over plants**: Ivy can manipulate plant growth, causing them to grow at an accelerated rate or manipulate their composition to create various toxins.
2. **Immunity to toxins**: Her exposure to toxins has granted her resistance, making her highly resilient to poisonous substances.
3. **Pheromone creation**: Ivy can produce pheromones that influence plant behavior, allowing her to control the growth and actions of flora.
4. **Superhuman strength**: Through her connection with plants, Ivy can augment her physical abilities, granting her enhanced strength.

**Weaknesses and Vulnerabilities**

While Poison Ivy's powers are formidable, they also present several weaknesses:

1. **Overwhelming passion for plants**: Her intense emotional investment in the natural world can cloud her judgment, leading to impulsive decisions that may compromise her operations.
2. **Vulnerability to fire**: Exposure to high temperatures or flames can temporarily disrupt her control over plants and leave her disoriented.
3. **Emotional connection to nature**: Ivy's deep affinity for the environment makes her particularly susceptible to emotional manipulation. This vulnerability can be exploited by adversaries, potentially undermining her resolve.

**Tactical Considerations**

When facing Poison Ivy, it is essential to consider her unique abilities and vulnerabilities:

1. **Exploit her passion**: Use environmental messaging or propaganda to fuel

In [31]:
# Interactive function to analyze any villain

def analyze_villain_interactive():
    """
    Interactive function to get villain information and generate Batman's analysis
    """
    print("Batman's Villain Intelligence Database")
    print("=" * 50)
    
    villain_name = input("Enter villain name: ")
    
    print("\nEnter villain information (press Enter twice when done):")
    print("Format: Work, Powers, Weaknesses")
    
    lines = []
    while True:
        line = input()
        if line == "":
            if lines:  # If we have content and get empty line, break
                break
        else:
            lines.append(line)
    
    villain_info = "\n".join(lines)
    
    print("\n" + "=" * 50)
    print("Generating Batman's analysis...")
    print("=" * 50 + "\n")
    
    display_batman_analysis(villain_name, villain_info)

# Uncomment to use interactively:
# analyze_villain_interactive()

In [32]:
# Function to process multiple villains from a dictionary

def batch_analyze_villains(villains_dict):
    """
    Analyze multiple villains and return all analyses
    
    Args:
        villains_dict: Dictionary with villain names as keys and info as values
    
    Returns:
        Dictionary with villain names and their Batman analyses
    """
    analyses = {}
    
    for villain_name, villain_info in villains_dict.items():
        print(f"Analyzing {villain_name}...")
        analysis = batman_summarize_villain(villain_name, villain_info)
        analyses[villain_name] = analysis
    
    return analyses

# Example usage:
# all_analyses = batch_analyze_villains(villains)
# for name, analysis in all_analyses.items():
#     print(f"\n{'='*60}")
#     print(f"## {name}")
#     print(f"{'='*60}")
#     display(Markdown(analysis))

In [33]:
# Quick test - make sure Ollama is running first!

test_villain = {
    "Scarecrow": """
    Work: Criminal psychologist who uses fear toxins
    Powers: Expert in psychology and chemistry, creates fear-inducing toxins, master of psychological warfare
    Weakness: Fear of Batman, overconfidence in his toxins, physical weakness
    """
}

display_batman_analysis("Scarecrow", test_villain["Scarecrow"])

## Batman's Analysis: Scarecrow

---



The Scarecrow: a nemesis whose modus operandi is predicated upon the manipulation of fear, a psychological toxin that can unravel even the most resolute individual. As I analyze this threat, I must acknowledge the complexity of his approach, which warrants a comprehensive tactical assessment.

**Criminal Operations and Methods**

The Scarecrow's primary objective is to instill fear into those who dare oppose him. His tactics are twofold: first, he uses his expertise in psychology and chemistry to develop toxin that induces extreme fear, often to the point of incapacitation. This toxin, known as Fear Neurotoxin (FNT), can be delivered through various means, including aerosol sprays, injections, or even airborne pathogens. His arsenal is augmented by psychological warfare, where he preys upon his victims' deepest phobias and anxieties.

**Powers and Capabilities**

The Scarecrow's powers lie in his ability to create and manipulate fear toxins, which grant him an unsettling advantage. As a master of psychology and chemistry, he can tailor his toxin to exploit specific vulnerabilities, making him a formidable opponent. His expertise in psychological warfare allows him to read people, anticipate their reactions, and adapt his tactics accordingly.

**Weaknesses and Vulnerabilities**

While the Scarecrow's powers are formidable, he is not invincible. Three key weaknesses have been identified:

1. **Fear of Batman**: The Scarecrow's greatest weakness lies in his own psychological vulnerability to me. His arrogance, coupled with a deep-seated insecurity, renders him susceptible to my determination and unwavering resolve.
2. **Overconfidence**: The Scarecrow's reliance on his toxins can lead him to underestimate the capabilities of others. This overconfidence often results in complacency, which I will exploit to neutralize his threat.
3. **Physical Weakness**: Despite his psychological prowess, the Scarecrow is a physically frail individual. His aging process has taken its toll, leaving him vulnerable to injuries and physical strains.

**Tactical Assessment**

Given these vulnerabilities, my strategy for countering the Scarecrow's threats would be as follows:

1. **Prevention**: Utilize advanced surveillance systems to track his movements and anticipate potential toxin releases.
2. **Deterrence**: Employ various tactics to disrupt his operations, such as deploying counter-agents or disrupting his toxin production facilities.
3. **Infiltration**: Infiltrate the Scarecrow's inner circle